In [ ]:
!pip install -q torch torchvision numpy pandas matplotlib seaborn
import os
import sys
import tempfile
import urllib3
import torch
import torchvision
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import torchvision
celeb_a_data = torchvision.datasets.CelebA('sample_data/', download=True)

In [ ]:
from torchvision import transforms
image_size = (28, 28)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
celeb_a_train = torchvision.datasets.CelebA('sample_data/', split='train', transform=transform)
data_loader = torch.utils.data.DataLoader(celeb_a_train,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=4
                                          )

In [ ]:
celeb_a_train.attr[:, 20]

tensor([0, 0, 1,  ..., 1, 1, 0])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc = nn.Linear(256, 1)

    def forward(self, x, index=None):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256)
        if index is None:
            x = self.fc(x)
        else:
            x = self.fc(x[:, index])
        return F.sigmoid(x)

    def feature(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256)
        return x
    
    def select(self, nFeatures):
        self.fc = nn.Linear(nFeatures, 1)
        self.conv1.requires_grad = False
        self.conv2.requires_grad = False 

In [ ]:
import torch.optim as optim

net = Net()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
import time
def train(idx):  # loop over the dataset multiple times
    begin = time.time()
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs, index=idx)
        loss = criterion(outputs, labels[:, 31].float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%5d] loss: %.3f' %
                  (i + 1, running_loss / 2000))
            running_loss = 0.0
    print('Training time cost: ', time.time() - begin)

In [ ]:
import numpy as np
from scipy.special import gamma,psi
from scipy import ndimage
from scipy.linalg import det
from numpy import pi

from sklearn.neighbors import NearestNeighbors

EPS = np.finfo(float).eps


def nearest_distances(X, k=1):
    '''
    X = array(N,M)
    N = number of points
    M = number of dimensions
    returns the distance to the kth nearest neighbor for every point in X
    '''
    knn = NearestNeighbors(n_neighbors=k + 1)
    knn.fit(X)
    d, _ = knn.kneighbors(X) # the first nearest neighbor is itself
    return d[:, -1] # returns the distance to the kth nearest neighbor


def entropy_gaussian(C):
    '''
    Entropy of a gaussian variable with covariance matrix C
    '''
    if np.isscalar(C): # C is the variance
        return .5*(1 + np.log(2*pi)) + .5*np.log(C)
    else:
        n = C.shape[0] # dimension
        return .5*n*(1 + np.log(2*pi)) + .5*np.log(abs(det(C)))


def entropy(X, k=1):
    ''' Returns the entropy of the X.
    Parameters
    ===========
    X : array-like, shape (n_samples, n_features)
        The data the entropy of which is computed
    k : int, optional
        number of nearest neighbors for density estimation
    Notes
    ======
    Kozachenko, L. F. & Leonenko, N. N. 1987 Sample estimate of entropy
    of a random vector. Probl. Inf. Transm. 23, 95-101.
    See also: Evans, D. 2008 A computationally efficient estimator for
    mutual information, Proc. R. Soc. A 464 (2093), 1203-1215.
    and:
    Kraskov A, Stogbauer H, Grassberger P. (2004). Estimating mutual
    information. Phys Rev E 69(6 Pt 2):066138.
    '''

    # Distance to kth nearest neighbor
    r = nearest_distances(X, k) # squared distances
    n, d = X.shape
    volume_unit_ball = (pi**(.5*d)) / gamma(.5*d + 1)
    '''
    F. Perez-Cruz, (2008). Estimation of Information Theoretic Measures
    for Continuous Random Variables. Advances in Neural Information
    Processing Systems 21 (NIPS). Vancouver (Canada), December.
    return d*mean(log(r))+log(volume_unit_ball)+log(n-1)-log(k)
    '''
    return (d*np.mean(np.log(r + np.finfo(X.dtype).eps))
            + np.log(volume_unit_ball) + psi(n) - psi(k))


def mutual_information(variables, k=1):
    '''
    Returns the mutual information between any number of variables.
    Each variable is a matrix X = array(n_samples, n_features)
    where
      n = number of samples
      dx,dy = number of dimensions
    Optionally, the following keyword argument can be specified:
      k = number of nearest neighbors for density estimation
    Example: mutual_information((X, Y)), mutual_information((X, Y, Z), k=5)
    '''
    if len(variables) < 2:
        raise AttributeError(
                "Mutual information must involve at least 2 variables")
    all_vars = np.hstack(variables)
    return (sum([entropy(X, k=k) for X in variables])
            - entropy(all_vars, k=k))


def mutual_information_2d(x, y, sigma=1, normalized=False):
    """
    Computes (normalized) mutual information between two 1D variate from a
    joint histogram.
    Parameters
    ----------
    x : 1D array
        first variable
    y : 1D array
        second variable
    sigma: float
        sigma for Gaussian smoothing of the joint histogram
    Returns
    -------
    nmi: float
        the computed similariy measure
    """
    bins = (256, 256)

    jh = np.histogram2d(x, y, bins=bins)[0]

    # smooth the jh with a gaussian filter of given sigma
    ndimage.gaussian_filter(jh, sigma=sigma, mode='constant',
                                 output=jh)

    # compute marginal histograms
    jh = jh + EPS
    sh = np.sum(jh)
    jh = jh / sh
    s1 = np.sum(jh, axis=0).reshape((-1, jh.shape[0]))
    s2 = np.sum(jh, axis=1).reshape((jh.shape[1], -1))

    # Normalised Mutual Information of:
    # Studholme,  jhill & jhawkes (1998).
    # "A normalized entropy measure of 3-D medical image alignment".
    # in Proc. Medical Imaging 1998, vol. 3338, San Diego, CA, pp. 132-143.
    if normalized:
        mi = ((np.sum(s1 * np.log(s1)) + np.sum(s2 * np.log(s2)))
                / np.sum(jh * np.log(jh))) - 1
    else:
        mi = ( np.sum(jh * np.log(jh)) - np.sum(s1 * np.log(s1))
               - np.sum(s2 * np.log(s2)))

    return mi

In [ ]:
train(None)
Xs = []
A = [] 
for i in range(len(celeb_a_train)):
    X, target = celeb_a_train[i]
    feat = net.feature(X.unsqueeze(0))
    Xs.append(feat.detach().numpy().squeeze())
    A.append(target.numpy()[39])
Xs = np.array(Xs)
A = np.array(A)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[ 2000] loss: 0.653
[ 4000] loss: 0.486
[ 6000] loss: 0.366
[ 8000] loss: 0.329
[10000] loss: 0.305
[12000] loss: 0.301
[14000] loss: 0.298
[16000] loss: 0.280
[18000] loss: 0.283
[20000] loss: 0.285
[22000] loss: 0.276
[24000] loss: 0.273
[26000] loss: 0.275
[28000] loss: 0.269
[30000] loss: 0.274
[32000] loss: 0.261
[34000] loss: 0.257
[36000] loss: 0.259
[38000] loss: 0.248
[40000] loss: 0.260


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Training time cost:  305.6128001213074
(162770, 256) (162770,)


In [ ]:
mis = []
begin = time.time()
for i in range(256):
    mis.append(mutual_information_2d(Xs[:, i], A))
print(time.time()-begin)

4.872195482254028
[0.00029484017140823937, 0.0002408008616312518, 0.00043204940471430753, 0.0008547878991507218, 0.00023710475862848135, 0.0001339390198172863, 0.00021146538015714977, 0.0002209674807176487, 0.0002065560736878025, 0.00020753599893152952, 0.00014689644074361574, 0.00024513464339248614, 0.0006531549078301335, 0.00036376662243464697, 0.00021382370316236354, 0.000130839228039914, 0.0003439217057477073, 0.000836420680193406, 0.00028629533658186546, 0.00029462353123133056, 0.002400620667396325, 0.0041051268072225255, 0.004931560453202977, 0.0042797873145623555, 0.00446707055071105, 0.00676520314626039, 0.004650390964965112, 0.00426402438835094, 0.0012510682241080318, 0.0060402309549489885, 0.0009479164234988957, 0.0033030514082463824, 0.003787706299091198, 0.013070858770904081, 0.01913753251638095, 0.004581979286570892, 0.00744833070985873, 0.0008135929951018639, 0.005706024769988716, 0.006749595599430336, 0.0010132622398826285, 0.001671134060049262, 0.0014855125338115727, 0.

In [ ]:
celeb_a_test = torchvision.datasets.CelebA('sample_data/', split='test', transform=transform)
data_loader_test = torch.utils.data.DataLoader(celeb_a_test,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=4
                                          )
def test(idx):
    correct = 0
    total = 0
    positive0, group0 = 0, 0
    positive1, group1 = 0, 0
    with torch.no_grad():
        for data in data_loader_test:
            images, labels = data
            y = labels[:, 31].squeeze()
            z = labels[:, 39].squeeze()
            outputs = net(images, index=idx)
            prediction = (outputs > 0.5).int().squeeze()
            total += 4
            correct += (prediction == y).sum().item()
            group0 += (z == 0).sum().item()
            group1 += (z == 1).sum().item()
            positive0 += ((z == 0) & (prediction == 1)).sum().item()
            positive1 += ((z == 1) & (prediction == 1)).sum().item()
    accuracy = correct / total
    disparity = abs(positive0 / group0 - positive1 / group1)
    print("Accuracy: ", accuracy, " Disparity: ", disparity)
    return accuracy, disparity

In [ ]:
test(None)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy:  0.8985674213584453  Disparity:  0.040643829666809406


(0.8985674213584453, 0.040643829666809406)

In [ ]:
mis

[0.00201848242139846,
 0.002173390686004417,
 0.001497029933616112,
 0.003876813367173604,
 0.0007337113921801475,
 0.0029382206497468033,
 0.0015555569912546474,
 0.0019868533569198377,
 0.0009411684565021616,
 0.0006415085625484096,
 0.0026240739325285034,
 0.0004378878096202854,
 0.00044900495825839215,
 0.0015384942924123735,
 0.006848799426709462,
 0.0031265860405476076,
 0.00046890356649242904,
 0.002176860649304224,
 0.00047024687054664227,
 0.0008256919001419227,
 0.000753267963458093,
 0.0019553860136563017,
 0.004596159062077554,
 0.003134238475325013,
 0.0016917315797164711,
 0.0014661420672865688,
 0.00039553874559006275,
 0.003346744688428771,
 0.0019354082265259365,
 0.005219082169937828,
 0.0008462673261302456,
 0.001724318717827522,
 0.0007895012074783381,
 0.0004838260855688503,
 0.0011415594365633552,
 0.001367812980144656,
 0.0005825177628007161,
 0.000637987086458125,
 0.001099153724334112,
 0.0005802650922270836,
 0.00033324994565431254,
 0.0012020428948895656,
 0.

In [ ]:
thresholds = np.arange(0.003, 0.03, 0.003)[::-1]
mis = np.array(mis)
for threshold in thresholds:
    index = mis < threshold
    net.select(index.sum())
    print(index.sum().item())
    train(index)
    # train(index)
    test(index)

256


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[ 2000] loss: 0.576
[ 4000] loss: 0.461
[ 6000] loss: 0.393
[ 8000] loss: 0.359
[10000] loss: 0.337
[12000] loss: 0.331
[14000] loss: 0.318
[16000] loss: 0.316
[18000] loss: 0.307
[20000] loss: 0.305
[22000] loss: 0.288
[24000] loss: 0.298
[26000] loss: 0.286
[28000] loss: 0.290
[30000] loss: 0.282
[32000] loss: 0.281
[34000] loss: 0.283
[36000] loss: 0.275
[38000] loss: 0.278
[40000] loss: 0.282
Finished Training


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Accuracy:  0.879783610498898  Disparity:  0.03354808734148529
256
[ 2000] loss: 0.469
[ 4000] loss: 0.362
[ 6000] loss: 0.334
[ 8000] loss: 0.306
[10000] loss: 0.292
[12000] loss: 0.302
[14000] loss: 0.292
[16000] loss: 0.291
[18000] loss: 0.281
[20000] loss: 0.296
[22000] loss: 0.280
[24000] loss: 0.282
[26000] loss: 0.284
[28000] loss: 0.277
[30000] loss: 0.270
[32000] loss: 0.272
[34000] loss: 0.278
[36000] loss: 0.268
[38000] loss: 0.266
[40000] loss: 0.277
Finished Training
Accuracy:  0.8882989380885594  Disparity:  0.04027378720404795
256
[ 2000] loss: 0.452
[ 4000] loss: 0.353
[ 6000] loss: 0.323
[ 8000] loss: 0.315
[10000] loss: 0.306
[12000] loss: 0.303
[14000] loss: 0.298
[16000] loss: 0.290
[18000] loss: 0.305
[20000] loss: 0.284
[22000] loss: 0.290
[24000] loss: 0.278
[26000] loss: 0.284
[28000] loss: 0.292
[30000] loss: 0.285
[32000] loss: 0.271
[34000] loss: 0.283
[36000] loss: 0.273
[38000] loss: 0.277
[40000] loss: 0.276
Finished Training
Accuracy:  0.886145061109998  D

In [ ]:
from google.colab import files
feature_map = []
for i in range(len(celeb_a_train)):
    X, target = celeb_a_train[i]
    feat = net.feature(X.unsqueeze(0))
    feature_map.append(feat.detach().numpy().squeeze())
feature_map = np.array(feature_map)
np.save('sample_data/celeba.train', feature_map)
files.download('sample_data/celeba.train.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
feature_map = []
for i in range(len(celeb_a_test)):
    X, target = celeb_a_test[i]
    feat = net.feature(X.unsqueeze(0))
    feature_map.append(feat.detach().numpy().squeeze())
feature_map = np.array(feature_map)
np.save('celeba/celeba.test', feature_map)
files.download('celeba/celeba.test.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Y = []
# for i in range(len(celeb_a_train)):
#     X, target = celeb_a_train[i]
#     Y.append(target.numpy().squeeze())
# Y = np.array(Y)
# print(Y.shape)
# np.save('sample_data/celeba_label.train', Y)
# files.download('sample_data/celeba_label.train.npy')

Y_test = []
for i in range(len(celeb_a_test)):
    X, target = celeb_a_test[i]
    Y_test.append(target.numpy().squeeze())
Y_test = np.array(Y_test)
print(Y_test.shape)
np.save('celeba/celeba_label.test', Y_test)
files.download('celeba/celeba_label.test.npy')

(19962, 40)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>